In [14]:
import logging,os,sys
from torch.utils.data import Dataset, DataLoader
sys.path.append('/root/workspace/wht/multimodal_brain/src')
from dataset.things_eeg import EEGDataset # type: ignore
import torch
from models.mlp import MLP,ProjectLayer,Direct
from scipy.spatial.distance import cdist
import numpy as np

In [3]:
selected_ch = ['Fp1', 'Fp2', 'AF7', 'AF3', 'AFz', 'AF4', 'AF8', 'F7', 'F5', 'F3',
                                            'F1', 'F2', 'F4', 'F6', 'F8', 'FT9', 'FT7', 'FC5', 'FC3', 'FC1', 
                                            'FCz', 'FC2', 'FC4', 'FC6', 'FT8', 'FT10', 'T7', 'C5', 'C3', 'C1',
                                            'Cz', 'C2', 'C4', 'C6', 'T8', 'TP9', 'TP7', 'CP5', 'CP3', 'CP1', 
                                            'CPz', 'CP2', 'CP4', 'CP6', 'TP8', 'TP10', 'P7', 'P5', 'P3', 'P1',
                                            'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8',
                                            'O1', 'Oz', 'O2']
latend_dim = 512
config = {
    "data_dir": "/root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten",#"/dev/shm/wht/datasets/things-eeg-small/Preprocessed_data_250Hz/",
    "exp_root":'./exp',
    "name": 'subject_semantic',
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "lr": 1e-4,
    "epochs": 100,
    "batch_size": 1024,
    "model_type":'ViT-B-32', #ViT-bigG-14,ViT-H-14
    "logger": True,
    "subjects":['sub-01','sub-02','sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10'],
    "eeg_model":{'name':'MLP','args':{'input_dim':len(selected_ch)*250,'output_dim':latend_dim,'hiden_dims':[512]}},#{'name':'LSTMModel', 'args':{}},
    "img_model": {'name':'Direct','args':{}},  # {'name':'ProjectLayer', 'args':{'embedding_dim':1024, 'proj_dim':1024}},
    "text_model": {'name':'Direct','args':{}},  # {'name':'ProjectLayer', 'args':{'embedding_dim':1024, 'proj_dim':1024}},
}
test_dataset = EEGDataset(data_dir=config['data_dir'],subjects=config['subjects'],model_type=config['model_type'],mode='test',transform=None,avg=True)

logging.info(f"test num: {len(test_dataset)}")
test_loader = DataLoader(test_dataset, batch_size=200, shuffle=False, drop_last=False,num_workers=4)
data={
    'test':test_loader
}

----load /root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/sub-01/test.pt----
----load /root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/sub-02/test.pt----
----load /root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/sub-03/test.pt----
----load /root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/sub-04/test.pt----
----load /root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/sub-05/test.pt----
----load /root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/sub-06/test.pt----
----load /root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/sub-07/test.pt----
----load /root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz_whiten/sub-08/test.pt----
----load /root/workspace

In [4]:
eeg_model = globals()[config['eeg_model']['name']](**config['eeg_model']['args'])
eeg_model.to(config['device'])
logging.info(f'eeg_model:{eeg_model}')
eeg_model.load_state_dict(torch.load(os.path.join('../',config['exp_root'],'train_multi_subject',f"ckpt_{'_'.join(config['subjects'])}.pth"),map_location=config['device'])['eeg'])

<All keys matched successfully>

In [12]:
# 
i = 0
eeg = test_dataset.loaded_data[i]['eeg']
eeg = eeg.to(config['device']).to(torch.float32)

latent = eeg_model(eeg)
latent.shape

torch.Size([200, 512])

In [13]:
latent_np = latent.cpu().detach().numpy()
distance_matrix = cdist(latent_np, latent_np, metric='euclidean')
print(distance_matrix.shape)

(200, 200)


In [16]:
average_distance = np.mean(distance_matrix)
print(average_distance)

11.781839204052341


In [32]:
_ = []
for i in range(10):
    eeg = test_dataset.loaded_data[i]['eeg']
    eeg = eeg.to(config['device']).to(torch.float32)
    
    latent = eeg_model(eeg) 
    
    latent = latent.unsqueeze(0)
    _.append(latent.cpu().detach().numpy())

_ = np.concatenate(_)
print(_.shape)

avg = []
for index in range(200):
    l = _[:,index]
    distance_matrix = cdist(l, l, metric='euclidean')
    average_distance = np.mean(distance_matrix)
    avg.append(average_distance)
print(np.mean(avg))

(10, 200, 512)
9.604742810417926
